In this notebook, we demo implementation and performance on OpenOOD Benchmark [1]

Ref: [1] https://github.com/Jingkang50/OpenOOD/tree/main

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 28 04:07:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

1. Install OpenOOD with pip and make necessary preparation

In [2]:
!pip show OpenOOD

In [3]:
!pip install git+https://github.com/Jingkang50/OpenOOD

  Cloning https://github.com/Jingkang50/OpenOOD to /tmp/pip-req-build-7yz7l7p3
  Running command git clone --filter=blob:none --quiet https://github.com/Jingkang50/OpenOOD /tmp/pip-req-build-7yz7l7p3
  Resolved https://github.com/Jingkang50/OpenOOD to commit 18c6f5174a2f518e2a8e819ffb1cd1914bcf12e0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86

In [4]:
# necessary imports
import torch

from openood.evaluation_api import Evaluator
from openood.networks import ResNet18_32x32 #just a wrapper around the ResNet
from openood.networks.ash_net import ASHNet

In [5]:
# download our pre-trained CIFAR-10 classifier
!gdown 1byGeYxM_PlLjT72wZsMQvP6popJeWBgt
!unzip cifar10_res18_v1.5.zip

Downloading...
From (original): https://drive.google.com/uc?id=1byGeYxM_PlLjT72wZsMQvP6popJeWBgt
From (redirected): https://drive.google.com/uc?id=1byGeYxM_PlLjT72wZsMQvP6popJeWBgt&confirm=t&uuid=af74a0b7-fa53-4990-b12d-5fe08edf39a8
To: /content/cifar10_res18_v1.5.zip
100% 375M/375M [00:09<00:00, 38.6MB/s]
Archive:  cifar10_res18_v1.5.zip
   creating: cifar10_resnet18_32x32_base_e100_lr0.1_default/
   creating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s2/
  inflating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s2/best_epoch99_acc0.9450.ckpt  
  inflating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s2/config.yml  
  inflating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s2/best.ckpt  
  inflating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s2/last_epoch100_acc0.9420.ckpt  
  inflating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s2/log.txt  
   creating: cifar10_resnet18_32x32_base_e100_lr0.1_default/s1/
  inflating: cifar10_resnet18_32x32_base_e100_lr0.1_default

In [6]:
# load the model
net = ResNet18_32x32(num_classes=10)
net.load_state_dict(
    torch.load('./cifar10_resnet18_32x32_base_e100_lr0.1_default/s0/best.ckpt')
)
net.cuda()
net.eval()

ResNet18_32x32(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1,

2. SOTA Baseline

In [7]:
knn_evaluator = Evaluator(
    net,
    id_name='cifar10',                     # the target ID dataset
    data_root='./data',                    # change if necessary
    config_root=None,                      # see notes above
    preprocessor=None,                     # default preprocessing for the target ID dataset
    postprocessor_name='knn',              # the postprocessor to use
    postprocessor=None,                    # if you want to use your own postprocessor
    batch_size=200,                        # for certain methods the results can be slightly affected by batch size
    shuffle=False,
    num_workers=2)                         # could use more num_workers outside colab

Downloading...
From (original): https://drive.google.com/uc?id=1XKzBdWCqg3vPoj-D32YixJyJJ0hL63gP
From (redirected): https://drive.google.com/uc?id=1XKzBdWCqg3vPoj-D32YixJyJJ0hL63gP&confirm=t&uuid=a20bab61-f7ec-484a-a460-44409751cea2
To: /content/data/benchmark_imglist.zip
100%|██████████| 27.7M/27.7M [00:01<00:00, 18.7MB/s]


cifar10 needs download:
./data/images_classic/cifar10


Downloading...
From (original): https://drive.google.com/uc?id=1Co32RiiWe16lTaiOU6JMMnyUYS41IlO1
From (redirected): https://drive.google.com/uc?id=1Co32RiiWe16lTaiOU6JMMnyUYS41IlO1&confirm=t&uuid=5178b001-0fab-422c-9259-d2c6b4bffb51
To: /content/data/images_classic/cifar10/cifar10.zip
100%|██████████| 143M/143M [00:02<00:00, 53.3MB/s]


cifar100 needs download:
./data/images_classic/cifar100


Downloading...
From (original): https://drive.google.com/uc?id=1PGKheHUsf29leJPPGuXqzLBMwl8qMF8_
From (redirected): https://drive.google.com/uc?id=1PGKheHUsf29leJPPGuXqzLBMwl8qMF8_&confirm=t&uuid=5d396f46-8a73-4301-8ba8-491b5ef054e9
To: /content/data/images_classic/cifar100/cifar100.zip
100%|██████████| 141M/141M [00:01<00:00, 80.7MB/s]


...

Hyperparam: [200], auroc: 0.902765


100%|██████████| 5/5 [00:02<00:00,  2.06it/s]


Hyperparam: [500], auroc: 0.9000844999999998


100%|██████████| 5/5 [00:03<00:00,  1.50it/s]

Hyperparam: [1000], auroc: 0.8976035
Final hyperparam: 50


In [8]:
metrics = knn_evaluator.eval_ood(fsood=False)

Performing inference on cifar10 test set...


100%|██████████| 45/45 [00:19<00:00,  2.26it/s]

Processing near ood...
Performing inference on cifar100 dataset...



100%|██████████| 45/45 [00:18<00:00,  2.49it/s]

Computing metrics on cifar100 dataset...
FPR@95: 37.91, AUROC: 89.75 AUPR_IN: 90.11, AUPR_OUT: 88.37
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 24.56, AUROC: 93.02 AUPR_IN: 96.04, AUPR_OUT: 87.10
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 176/176 [01:45<00:00,  1.68it/s]

Computing metrics on places365 dataset...
FPR@95: 29.50, AUROC: 92.10 AUPR_IN: 81.27, AUPR_OUT: 97.37
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 23.88, AUROC: 93.11 AUPR_IN: 87.26, AUPR_OUT: 94.92
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:09<00:00,  4.88it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar100    37.91  89.75    90.11     88.37 95.22
tin         31.18  91.65    93.35     89.06 95.22
nearood     34.54  90.70    91.73     88.71 95.22
mnist       20.62  94.41    82.75     99.01 95.22
svhn        20.83  92.89    88.98     96.21 95.22
texture     24.56  93.02    96.04     87.10 95.22
places365   29.50  92.10    81.27     97.37 95.22
farood      23.88  93.11    87.26     94.92 95.22


5. fDBD Implementation and Evaluation

In [16]:
import numpy as np
for i, param in enumerate(net.fc.parameters()):
  if i == 0:
    w = param.data.cpu().numpy()
  else:
    b = param.data.cpu().numpy()

In [17]:
denominator_matrix = np.zeros((10,10))
for p in range(10):
  w_p = w - w[p,:]
  denominator = np.linalg.norm(w_p, axis=1)
  denominator[p] = 1
  denominator_matrix[p, :] = denominator

denominator_matrix = torch.tensor(denominator_matrix).cuda()

In [18]:
train_mean = np.mean(knn_evaluator.postprocessor.activation_log, axis=0)
train_mean_tensor = torch.from_numpy(train_mean).cuda()

In [19]:
from typing import Any

import torch
import torch.nn as nn

from openood.postprocessors import BasePostprocessor

class fDBDPostprocessor(BasePostprocessor):
    def __init__(self, config):
        super(fDBDPostprocessor, self).__init__(config)
        self.APS_mode = False

    @torch.no_grad()
    def postprocess(self, net: nn.Module, data: Any):
        output, feature = net(data, return_feature=True)
        values, nn_idx = output.max(1)
        logits_sub = torch.abs(output - values.repeat(10, 1).T)
        score = torch.sum(logits_sub/denominator_matrix[nn_idx], axis=1)/torch.norm(feature - train_mean_tensor , dim = 1)
        return nn_idx, score

In [20]:
fdbd_evaluator = Evaluator(
    net,
    id_name='cifar10',                     # the target ID dataset
    data_root='./data',                    # change if necessary
    config_root=None,                      # see notes above
    preprocessor=None,                     # default preprocessing for the target ID dataset
    #postprocessor_name=postprocessor_name, # the postprocessor to use
    postprocessor= fDBDPostprocessor('./configs'),       # if you want to use your own postprocessor
    batch_size=200,                        # for certain methods the results can be slightly affected by batch size
    shuffle=False,
    num_workers=2)                         # could use more num_workers outside colab

In [21]:
metrics = fdbd_evaluator.eval_ood(fsood=False)

Performing inference on cifar10 test set...


100%|██████████| 45/45 [00:12<00:00,  3.69it/s]

Processing near ood...
Performing inference on cifar100 dataset...



100%|██████████| 45/45 [00:06<00:00,  6.97it/s]

Computing metrics on cifar100 dataset...
FPR@95: 41.14, AUROC: 89.36 AUPR_IN: 88.80, AUPR_OUT: 88.26
──────────────────────────────────────────────────────────────────────

Performing inference on tin dataset...


...

Computing metrics on texture dataset...
FPR@95: 24.98, AUROC: 92.83 AUPR_IN: 95.65, AUPR_OUT: 87.28
──────────────────────────────────────────────────────────────────────

Performing inference on places365 dataset...



100%|██████████| 176/176 [01:07<00:00,  2.61it/s]

Computing metrics on places365 dataset...
FPR@95: 26.40, AUROC: 92.70 AUPR_IN: 82.24, AUPR_OUT: 97.60
──────────────────────────────────────────────────────────────────────

Computing mean metrics...
FPR@95: 23.96, AUROC: 93.00 AUPR_IN: 85.23, AUPR_OUT: 94.97
──────────────────────────────────────────────────────────────────────




ID Acc Eval: 100%|██████████| 45/45 [00:06<00:00,  6.66it/s]

           FPR@95  AUROC  AUPR_IN  AUPR_OUT   ACC
cifar100    41.14  89.36    88.80     88.26 95.22
tin         31.71  91.48    92.49     89.26 95.22
nearood     36.43  90.42    90.64     88.76 95.22
mnist       20.28  94.59    78.55     99.10 95.22
svhn        24.18  91.89    84.48     95.90 95.22
texture     24.98  92.83    95.65     87.28 95.22
places365   26.40  92.70    82.24     97.60 95.22
farood      23.96  93.00    85.23     94.97 95.22
